In [8]:
# Import necessary packages
using Oceananigans
using Oceananigans.Units: hour, minute
using Oceananigans.Advection
using Oceananigans.OutputWriters: JLD2OutputWriter, TimeInterval
using Plots
using JLD2

# Define simulation parameters
Nx, Ny, Nz = 128, 128, 64          # Number of grid points
Lx, Ly, Lz = 1e4, 1e4, 1e3         # Domain size in meters
Δx, Δy, Δz = Lx/Nx, Ly/Ny, Lz/Nz   # Grid spacing

# Create a grid
grid = RectilinearGrid(size=(Nx, Ny, Nz), extent=(Lx, Ly, Lz))

# Physical constants
g = 9.81           # Gravitational acceleration (m/s^2)
ρ₀ = 1025.0        # Reference density (kg/m³)

# Initial condition: Density stratification with a perturbation
N² = 1e-4          # Buoyancy frequency squared (s⁻²)
α = 2e-4           # Thermal expansion coefficient (1/K)
ΔT = -N² * Lz / (g * α)  # Temperature difference across the domain

# Background temperature profile
θ_b(z) = ΔT * z / Lz

# Temperature perturbation to create internal waves
A = 0.01          # Amplitude of perturbation
k = 2π / (Lx / 4) # Wave number

θ₀(x, y, z) = θ_b(z) + A * sin(k * x) * sin(k * y) * exp(-(z - Lz / 2)^2 / (2 * (Lz / 10)^2))

# Create the model
model = HydrostaticFreeSurfaceModel(
    grid = grid,
    advection = WENO(),    # Adaptive-order WENO scheme
    timestepper = :RungeKutta3,
    tracers = (:T,),
    buoyancy = BuoyancyTracer(),
    closure = IsotropicDiffusivity(ν=0, κ=0),
    free_surface = FreeSurface(numerical_method=:implicit),
)

# Set initial conditions
set!(model, T=θ₀)

# Time-stepping parameters
Δt = 10.0         # Time step (seconds)
simulation_time = 2hour
n_steps = Int(simulation_time / Δt)

# Output writers
output_interval = 10minute
output_iterations = Int(output_interval / Δt)

field_slicer = FieldSlicer(k=Nz ÷ 2)  # Mid-depth slice

T_writer = JLD2OutputWriter(model, fields=("T"=>model.tracers.T),
                            prefix="results/T", schedule=TimeInterval(output_interval),
                            field_slicer=field_slicer)

# Initialize simulation
simulation = Simulation(model, Δt=Δt, stop_time=simulation_time, output_writers=(T_writer,))

# Run the simulation
@info "Starting simulation..."
run!(simulation)
@info "Simulation complete!"

# Visualization
using Plots

# Load and plot the results
iterations = collect(T_writer.schedule.iterations)

for (i, iteration) in enumerate(iterations)
    # Construct the filename manually
    filename = T_writer.prefix * "_" * lpad(iteration, 010, "0") * ".jld2"
    if isfile(filename)
        data = jldopen(filename, "r") do file
            file["T"][:,:]
        end

        x = model.grid.xC
        y = model.grid.yC

        p = contourf(x, y, data', levels=20, color=:viridis)
        xlabel!("x (m)")
        ylabel!("y (m)")
        time = iteration * Δt
        title!("Temperature at z = $(Lz/2) m, time = $(round(time/60; digits=2)) minutes")
        savefig("plots/temperature_$(lpad(i, 3, '0')).png")
    else
        @warn "File $filename does not exist."
    end
end

# Create an animation
@info "Creating animation..."
animation = @animate for (i, iteration) in enumerate(iterations)
    filename = T_writer.prefix * "_" * lpad(iteration, 010, "0") * ".jld2"
    if isfile(filename)
        data = jldopen(filename, "r") do file
            file["T"][:,:]
        end

        x = model.grid.xC
        y = model.grid.yC

        contourf(x, y, data', levels=20, color=:viridis)
        xlabel!("x (m)")
        ylabel!("y (m)")
        time = iteration * Δt
        title!("Temperature at z = $(Lz/2) m, time = $(round(time/60; digits=2)) minutes")
    else
        @warn "File $filename does not exist."
    end
end

gif(animation, "temperature_evolution.gif", fps=5)
@info "Animation saved as temperature_evolution.gif"


UndefVarError: UndefVarError: `IsotropicDiffusivity` not defined

In [4]:
import Pkg; Pkg.add("JLD2")

    Updating registry at `C:\Users\dadoi\.julia\registries\General.toml`
   Resolving package versions...
   Installed JLD2 ─ v0.4.53
    Updating `C:\Users\dadoi\.julia\environments\v1.10\Project.toml`
⌅ [033835bb] + JLD2 v0.4.53
    Updating `C:\Users\dadoi\.julia\environments\v1.10\Manifest.toml`
⌅ [033835bb] ↑ JLD2 v0.4.48 ⇒ v0.4.53
        Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated -m`
Precompiling project...
  ✓ JLD2
  1 dependency successfully precompiled in 51 seconds. 467 already precompiled.
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version
